Τι Είναι το ProbLife (σύνοψη από το paper)
Κάθε κυψελίδα έχει τιμή πιθανότητας επιβίωσης μεταξύ 0.0 και 1.0

Οι κλασικοί κανόνες του Game of Life (π.χ. survive με 2-3 neighbors, birth με 3) τροποποιούνται ώστε η "ζωντάνια" να προκύπτει με πιθανότητα, όχι ντετερμινιστικά

Επηρεάζεται από το πλήθος των "ζωντανών" γειτόνων

In [1]:
!pip install ipywidgets --quiet

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.3 MB/s eta 0:00:00


In [2]:
def init_grid(rows, cols, alive_prob=0.3):
    # Τυχαία αρχικοποίηση: πιθανότητα ζωής σε κάθε κυψελίδα
    grid = np.random.rand(rows, cols)
    grid = np.where(grid < alive_prob, 1.0, 0.0)  # Ζωντανές ή νεκρές
    return grid


In [3]:
def compute_next_prob_grid(grid, birth_threshold=3, survive_min=2, survive_max=3):
    rows, cols = grid.shape
    new_grid = np.zeros_like(grid)

    for i in range(rows):
        for j in range(cols):
            # Γείτονες
            neighbors = grid[max(0, i-1):min(i+2, rows), max(0, j-1):min(j+2, cols)]
            total = np.sum(neighbors) - grid[i, j]  # αφαίρεσε το ίδιο κελί
            count = np.count_nonzero(neighbors) - (1 if grid[i, j] > 0 else 0)

            if grid[i, j] > 0:  # Ζωντανό
                if survive_min <= count <= survive_max:
                    new_grid[i, j] = np.clip(grid[i, j] + 0.1 * np.random.rand(), 0.0, 1.0)
                else:
                    new_grid[i, j] = np.clip(grid[i, j] - 0.2 * np.random.rand(), 0.0, 1.0)
            else:  # Νεκρό
                if count == birth_threshold:
                    new_grid[i, j] = np.random.rand()
    return new_grid


In [4]:
def plot_grid(grid, step):
    plt.figure(figsize=(6, 6))
    plt.imshow(grid, cmap='YlGnBu', vmin=0, vmax=1)
    plt.title(f"ProbLife - Γενιά {step}")
    plt.colorbar(label="Πιθανότητα ζωής")
    plt.axis('off')
    plt.show()


In [6]:
def run_problife_gui():
    rows_slider = widgets.IntSlider(min=10, max=100, step=5, value=30, description="Γραμμές")
    cols_slider = widgets.IntSlider(min=10, max=100, step=5, value=30, description="Στήλες")
    steps_slider = widgets.IntSlider(min=1, max=100, step=1, value=10, description="Βήματα")
    start_prob_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.3, description="Αρχ. πιθανότητα")
    button = widgets.Button(description="Εκκίνηση")
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            grid = init_grid(rows_slider.value, cols_slider.value, start_prob_slider.value)
            for step in range(steps_slider.value):
                plot_grid(grid, step)
                grid = compute_next_prob_grid(grid)

    button.on_click(on_click)

    display(widgets.VBox([
        rows_slider, cols_slider, steps_slider, start_prob_slider, button, output
    ]))

run_problife_gui()
